# Share Price Data Cleaning
The purpose of this notebook is to show how I imported, cleaned and processed data from [SimFin](https://www.simfin.com/en/fundamental-data-download/) for my Stock Price Data analysis.

I will:

[Import the data.](#import-data)
Once the data is imported, I will clean the data. This includes:
1. [Edit columns](#column-edits) and prune columns, if needed
1. [Handling missing values](#handling-missing-values). I will check for null values using `.isnull()` and fill them in using `.fillna()` 

3. [Check for duplicates](#check-for-duplicate-values). `.duplicated()` and removing the duplicates, by using `.drop_duplicates()`
4. [Converting data types](#converting-data-types) by using `.astype()`
5. [Filter data](#filter-data). Create a new dataframe containing only the tidy data
5. [Create a new CSV](#create-new-csv-file). It will be created from the new, filtered dataframe from 2022 to use in my analysis.

## Import data

In [41]:
# libraries
import pandas as pd
import re

In [42]:
%%time
shares = pd.read_csv('us-shareprices-daily.csv', delimiter=';')
len(shares)


CPU times: user 2.01 s, sys: 663 ms, total: 2.67 s
Wall time: 2.94 s


5322568

In [43]:
# Get a quick overview
shares.head()

,Ticker,SimFinId,Date,Open,High,Low,Close,Adj. Close,Volume,Dividend,Shares Outstanding
0,A,45846,2018-08-07,66.83,67.94,66.63,67.66,64.78,2829039,NaN,319000000.0
1,A,45846,2018-08-08,67.74,68.15,67.34,67.38,64.51,1682000,NaN,319000000.0
2,A,45846,2018-08-09,67.48,67.62,66.61,66.69,63.85,1727776,NaN,319000000.0
3,A,45846,2018-08-10,66.82,66.87,65.93,66.26,63.44,2166251,NaN,319000000.0
4,A,45846,2018-08-13,66.44,66.99,65.67,65.94,63.13,2989306,NaN,319000000.0


In [44]:
# Get detailed information about the DataFrame shares
shares.info(verbose=True, memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5322568 entries, 0 to 5322567
Data columns (total 11 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Ticker              object 
 1   SimFinId            int64  
 2   Date                object 
 3   Open                float64
 4   High                float64
 5   Low                 float64
 6   Close               float64
 7   Adj. Close          float64
 8   Volume              int64  
 9   Dividend            float64
 10  Shares Outstanding  float64
dtypes: float64(7), int64(2), object(2)
memory usage: 1013.4 MB


## Column Edits

Changed the column names to be easier to work with, removing uppercase letters and replacing spaces with underscores. Create a method that uses regex to remove any punctuation and built-in `lower()` function to change all the columns at once. 

In [45]:
def edit_column_names(df):
    col_dict = {}
    for col in df.columns:
        col_edit = col.lower()
        col_edit = re.sub(r'[^a-zA-Z0-9\s]', '', col_edit).replace(' ', '_')
        col_dict[col] = col_edit
    df = df.rename(columns=col_dict)
    return df

shares = edit_column_names(shares)

In [46]:
shares.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5322568 entries, 0 to 5322567
Data columns (total 11 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ticker              object 
 1   simfinid            int64  
 2   date                object 
 3   open                float64
 4   high                float64
 5   low                 float64
 6   close               float64
 7   adj_close           float64
 8   volume              int64  
 9   dividend            float64
 10  shares_outstanding  float64
dtypes: float64(7), int64(2), object(2)
memory usage: 446.7+ MB


## Handling missing values

Investigate how many records are there for each column and the percentages of missing values, if there are any.

In [47]:
# number of null values
shares.isnull().sum()

ticker                      0
simfinid                    0
date                        0
open                        0
high                        0
low                         0
close                       0
adj_close                   0
volume                      0
dividend              5288387
shares_outstanding     380181
dtype: int64

Determine the percentage of missing values in Dividend and Shares Outstanding column, to determine if they are above 60%.

In [48]:
print('Percentage of missing records: ')
missing_percentage = shares.isnull().sum() / len(shares) * 100
missing_percentage

Percentage of missing records: 


ticker                 0.000000
simfinid               0.000000
date                   0.000000
open                   0.000000
high                   0.000000
low                    0.000000
close                  0.000000
adj_close              0.000000
volume                 0.000000
dividend              99.357810
shares_outstanding     7.142812
dtype: float64

Since 99% of the data for the Dividend column is missing and I do not need the data in it for my analysis, I will go ahead and remove it from the dataframe. 

In [49]:
shares = shares.drop(columns='dividend', axis=0)


In [50]:
shares.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5322568 entries, 0 to 5322567
Data columns (total 10 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ticker              object 
 1   simfinid            int64  
 2   date                object 
 3   open                float64
 4   high                float64
 5   low                 float64
 6   close               float64
 7   adj_close           float64
 8   volume              int64  
 9   shares_outstanding  float64
dtypes: float64(6), int64(2), object(2)
memory usage: 406.1+ MB


The dividend column has been removed. Now, I will check to see how many missing values there are now.

In [51]:
shares.isnull().sum()

ticker                     0
simfinid                   0
date                       0
open                       0
high                       0
low                        0
close                      0
adj_close                  0
volume                     0
shares_outstanding    380181
dtype: int64

Since the Dividend column contained a lot of null values. I do not need it or the SimFinId for my analysis so I will define the columns I do want by adding them to a list that will use in the option, `usecols=required_cols`, and include it in the `read_csv` call for the final dataframe and when I create my future dataset imports. 

In [ ]:
# Define the required columns for the data
required_cols = ['Ticker', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Shares Outstanding']

In [52]:
# check if there are any empty values in the dataframe
tickers_nullshares = shares.loc[shares['shares_outstanding'].isnull(), 'ticker'].unique()
print(tickers_nullshares)

['AAC' 'AAWH' 'ABTI' ... 'ZI' 'ZIP' 'ZIVO']


I will leave the Shares Outstanding column alone for now and just keep the null values in place. 

In [53]:
# Check info to see if it is dropped
shares.info(verbose=True, memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5322568 entries, 0 to 5322567
Data columns (total 10 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ticker              object 
 1   simfinid            int64  
 2   date                object 
 3   open                float64
 4   high                float64
 5   low                 float64
 6   close               float64
 7   adj_close           float64
 8   volume              int64  
 9   shares_outstanding  float64
dtypes: float64(6), int64(2), object(2)
memory usage: 972.8 MB


## Check for duplicate values 

In [54]:
# shares.duplicated().sum()
shares.groupby(shares.columns.tolist(),as_index=False).size()

,ticker,simfinid,date,open,high,low,close,adj_close,volume,shares_outstanding,size
0,A,45846,2018-08-07,66.83,67.94,66.63,67.66,64.78,2829039,319000000.0,1
1,A,45846,2018-08-08,67.74,68.15,67.34,67.38,64.51,1682000,319000000.0,1
2,A,45846,2018-08-09,67.48,67.62,66.61,66.69,63.85,1727776,319000000.0,1
3,A,45846,2018-08-10,66.82,66.87,65.93,66.26,63.44,2166251,319000000.0,1
4,A,45846,2018-08-13,66.44,66.99,65.67,65.94,63.13,2989306,319000000.0,1
...,...,...,...,...,...,...,...,...,...,...,...
4942382,ZYXI,171401,2023-07-05,9.50,9.54,9.15,9.17,9.17,215455,36435000.0,1
4942383,ZYXI,171401,2023-07-06,9.02,9.18,8.93,9.01,9.01,191404,36435000.0,1
4942384,ZYXI,171401,2023-07-07,9.03,9.29,8.94,9.00,9.00,291326,36435000.0,1
4942385,ZYXI,171401,2023-07-10,9.00,9.23,8.99,9.18,9.18,148425,36435000.0,1


Determine if there are duplicates.

In [55]:
duplicates = shares[shares.duplicated()] 
print(duplicates)

Empty DataFrame
Columns: [ticker, simfinid, date, open, high, low, close, adj_close, volume, shares_outstanding]
Index: []


There are no duplicates in the dataset.

## Converting data types
Define what data types will be best so that they don't use too much memory.

In [56]:
shares.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5322568 entries, 0 to 5322567
Data columns (total 10 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ticker              object 
 1   simfinid            int64  
 2   date                object 
 3   open                float64
 4   high                float64
 5   low                 float64
 6   close               float64
 7   adj_close           float64
 8   volume              int64  
 9   shares_outstanding  float64
dtypes: float64(6), int64(2), object(2)
memory usage: 406.1+ MB


I tried to change the float64 and int64 down to float16 and int16, but this was too low and adversely affected the data so I reloaded the dataset again and found that float32 and int32, worked best. 

I will change the float and int datatypes to float32 and int32, respectively, and the tickers to category. I changed the Date column manually, but when I bring in the dataset again, I will use `parse_date()` option on the `read_csv` call.

In [57]:
for col in shares.columns:
    if shares[col].dtype == 'float64':
        shares[col] = shares[col].astype('float32')
    if shares[col].dtype == 'int64':
        shares[col] = shares[col].astype('int32')
    if shares[col].dtype == 'object' and shares[col].name == 'Ticker':
        shares[col] = shares[col].astype('category')
    if shares[col].name == 'date':
        shares[col] = shares[col].astype('datetime64[ns]')

shares.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5322568 entries, 0 to 5322567
Data columns (total 10 columns):
 #   Column              Dtype         
---  ------              -----         
 0   ticker              object        
 1   simfinid            int32         
 2   date                datetime64[ns]
 3   open                float32       
 4   high                float32       
 5   low                 float32       
 6   close               float32       
 7   adj_close           float32       
 8   volume              int32         
 9   shares_outstanding  float32       
dtypes: datetime64[ns](1), float32(6), int32(2), object(1)
memory usage: 243.6+ MB


The code above loops through the columns and changes the type. The memory usage dropped.

Run `.head()` and `.describe` again to see if changing the types did anything adversely to the data.

In [58]:
shares.head()

,ticker,simfinid,date,open,high,low,close,adj_close,volume,shares_outstanding
0,A,45846,2018-08-07,66.830002,67.940002,66.629997,67.660004,64.779999,2829039,319000000.0
1,A,45846,2018-08-08,67.739998,68.150002,67.339996,67.379997,64.510002,1682000,319000000.0
2,A,45846,2018-08-09,67.480003,67.620003,66.610001,66.690002,63.849998,1727776,319000000.0
3,A,45846,2018-08-10,66.820000,66.870003,65.930000,66.260002,63.439999,2166251,319000000.0
4,A,45846,2018-08-13,66.440002,66.989998,65.669998,65.940002,63.130001,2989306,319000000.0


In [59]:
shares.describe()

,simfinid,date,open,high,low,close,adj_close,volume,shares_outstanding
count,5.322568e+06,5322568,5.322568e+06,5.322568e+06,5.322568e+06,5.322568e+06,5.322568e+06,5.322568e+06,4.942387e+06
mean,4.779305e+06,2021-02-25 05:37:53.133722112,5.433233e+02,5.623159e+02,5.113192e+02,5.358393e+02,5.238233e+02,1.694702e+06,2.605400e+08
min,1.800000e+01,2018-08-07 00:00:00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-2.125784e+09,0.000000e+00
25%,4.983910e+05,2019-12-10 00:00:00,8.880000e+00,9.110000e+00,8.620000e+00,8.860000e+00,8.320000e+00,4.277800e+04,1.893059e+07
50%,1.108872e+06,2021-03-23 00:00:00,2.210000e+01,2.255000e+01,2.164000e+01,2.208000e+01,2.053000e+01,2.607500e+05,4.901730e+07
75%,1.038350e+07,2022-05-17 00:00:00,5.401000e+01,5.499000e+01,5.303000e+01,5.400000e+01,5.090000e+01,9.620678e+05,1.283296e+08
max,1.566500e+07,2023-07-11 00:00:00,1.200000e+07,1.200000e+07,1.200000e+07,1.200000e+07,1.200000e+07,2.146534e+09,1.121052e+12
std,5.219623e+06,NaN,4.522812e+04,4.700781e+04,4.163399e+04,4.443310e+04,4.440703e+04,1.609591e+07,5.484785e+09


A dictionary variable will be used in the `read_csv()` call to change datatype when I bring in the clean dataset and I will add an option on the `read_csv` function to change the Date column to the correct datetype when I bring in the data.

## Filter Data
Check data by running queries

In [60]:
# Pull data by Ticker
gme_stock_data = shares[shares['ticker'] == 'GME']
gme_stock_data

,ticker,simfinid,date,open,high,low,close,adj_close,volume,shares_outstanding
2053021,GME,44534,2018-08-07,3.730000,3.810000,3.710000,3.780000,3.470000,7813724,407600000.0
2053022,GME,44534,2018-08-08,3.780000,3.830000,3.750000,3.810000,3.500000,8715468,407600000.0
2053023,GME,44534,2018-08-09,3.810000,3.870000,3.750000,3.800000,3.490000,9370924,407600000.0
2053024,GME,44534,2018-08-10,3.790000,3.900000,3.770000,3.830000,3.520000,9666664,407600000.0
2053025,GME,44534,2018-08-13,3.840000,3.850000,3.780000,3.800000,3.490000,8403788,407600000.0
...,...,...,...,...,...,...,...,...,...,...
2054255,GME,44534,2023-07-05,24.639999,24.850000,23.790001,23.900000,23.900000,2268244,304500000.0
2054256,GME,44534,2023-07-06,23.520000,23.570000,22.820000,22.830000,22.830000,2390303,304500000.0
2054257,GME,44534,2023-07-07,22.969999,23.530001,22.670000,22.709999,22.709999,2447203,304500000.0
2054258,GME,44534,2023-07-10,22.610001,23.559999,22.000000,23.540001,23.540001,3318214,304500000.0


In [61]:
# see if the data type change on date was successful

shares['date'] = pd.to_datetime(shares['date'])

# Set a new dataframe with the filtered data for year 2022, reset index
stock_data_2022 = shares[shares['date'].dt.year == 2022].reset_index()

stock_data_2022

,index,ticker,simfinid,date,open,high,low,close,adj_close,volume,shares_outstanding
0,858,A,45846,2022-01-03,159.000000,159.440002,153.929993,156.479996,153.809998,1606323,302722656.0
1,859,A,45846,2022-01-04,155.490005,155.630005,149.699997,151.190002,148.610001,2233958,302722656.0
2,860,A,45846,2022-01-05,150.830002,153.100006,148.529999,148.600006,146.070007,2370529,302722656.0
3,861,A,45846,2022-01-06,148.850006,149.960007,145.580002,149.119995,146.580002,2298277,302722656.0
4,862,A,45846,2022-01-07,149.119995,149.729996,145.089996,145.149994,142.679993,2058658,302722656.0
...,...,...,...,...,...,...,...,...,...,...,...
1171069,5322433,ZYXI,171401,2022-12-23,13.630000,14.070000,13.630000,13.810000,13.810000,180392,38046000.0
1171070,5322434,ZYXI,171401,2022-12-27,14.020000,14.090000,13.670000,13.760000,13.760000,143701,38046000.0
1171071,5322435,ZYXI,171401,2022-12-28,13.690000,13.900000,13.630000,13.880000,13.880000,137809,38046000.0
1171072,5322436,ZYXI,171401,2022-12-29,13.950000,14.200000,13.810000,13.870000,13.870000,159746,38046000.0


Determine if my new dataframe contains only data from the year 2022. 

In [62]:
stock_data_2022['date'].agg(['min', 'max'])

min   2022-01-03
max   2022-12-30
Name: date, dtype: datetime64[ns]

I have successfully pull data from only 2022.

In [63]:
stock_data_2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1171074 entries, 0 to 1171073
Data columns (total 11 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   index               1171074 non-null  int64         
 1   ticker              1171074 non-null  object        
 2   simfinid            1171074 non-null  int32         
 3   date                1171074 non-null  datetime64[ns]
 4   open                1171074 non-null  float32       
 5   high                1171074 non-null  float32       
 6   low                 1171074 non-null  float32       
 7   close               1171074 non-null  float32       
 8   adj_close           1171074 non-null  float32       
 9   volume              1171074 non-null  int32         
 10  shares_outstanding  1127863 non-null  float32       
dtypes: datetime64[ns](1), float32(6), int32(2), int64(1), object(1)
memory usage: 62.5+ MB


Look at the first 10 records.

In [64]:
stock_data_2022.head(10)

,index,ticker,simfinid,date,open,high,low,close,adj_close,volume,shares_outstanding
0,858,A,45846,2022-01-03,159.000000,159.440002,153.929993,156.479996,153.809998,1606323,302722656.0
1,859,A,45846,2022-01-04,155.490005,155.630005,149.699997,151.190002,148.610001,2233958,302722656.0
2,860,A,45846,2022-01-05,150.830002,153.100006,148.529999,148.600006,146.070007,2370529,302722656.0
3,861,A,45846,2022-01-06,148.850006,149.960007,145.580002,149.119995,146.580002,2298277,302722656.0
4,862,A,45846,2022-01-07,149.119995,149.729996,145.089996,145.149994,142.679993,2058658,302722656.0
5,863,A,45846,2022-01-10,143.289993,145.309998,140.860001,145.160004,142.690002,2548145,302722656.0
6,864,A,45846,2022-01-11,145.000000,146.940002,143.809998,146.639999,144.139999,2028671,302722656.0
7,865,A,45846,2022-01-12,147.800003,150.389999,147.550003,149.509995,146.960007,2250847,302722656.0
8,866,A,45846,2022-01-13,149.460007,149.539993,144.850006,145.169998,142.690002,1741764,302722656.0
9,867,A,45846,2022-01-14,144.039993,145.149994,142.360001,144.679993,142.210007,2225442,302722656.0


I will drop the simfinid column, as I do not need it. I will be using the ticker as the id for my analysis. 

In [65]:
stock_data_2022 = stock_data_2022.drop(columns='simfinid', axis=0)

In [66]:
stock_data_2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1171074 entries, 0 to 1171073
Data columns (total 10 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   index               1171074 non-null  int64         
 1   ticker              1171074 non-null  object        
 2   date                1171074 non-null  datetime64[ns]
 3   open                1171074 non-null  float32       
 4   high                1171074 non-null  float32       
 5   low                 1171074 non-null  float32       
 6   close               1171074 non-null  float32       
 7   adj_close           1171074 non-null  float32       
 8   volume              1171074 non-null  int32         
 9   shares_outstanding  1127863 non-null  float32       
dtypes: datetime64[ns](1), float32(6), int32(1), int64(1), object(1)
memory usage: 58.1+ MB


Change the Date column back to an Object type, as it uses less memory.

In [67]:
from datetime import datetime
def convert_datetime(dt):
    return datetime.strftime(dt, '%Y-%m-%d') # Change it back to the original format it came in

stock_data_2022['date']= stock_data_2022['date'].apply(convert_datetime)

In [68]:
stock_data_2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1171074 entries, 0 to 1171073
Data columns (total 10 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   index               1171074 non-null  int64  
 1   ticker              1171074 non-null  object 
 2   date                1171074 non-null  object 
 3   open                1171074 non-null  float32
 4   high                1171074 non-null  float32
 5   low                 1171074 non-null  float32
 6   close               1171074 non-null  float32
 7   adj_close           1171074 non-null  float32
 8   volume              1171074 non-null  int32  
 9   shares_outstanding  1127863 non-null  float32
dtypes: float32(6), int32(1), int64(1), object(2)
memory usage: 58.1+ MB


## Create new CSV file

The code below will create a new CSV using my new dataframe, `stock_data_2022`. 

In [69]:
# Push the new dataframe and convert it to a CSV
stock_data_2022.to_csv('US_Share_Prices_2022.csv', index=False)

## My Notes 

Import and inspect new file to make sure that it contains the data that I need.

In [ ]:
# Define the required columns for the data
required_cols = ['Ticker', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Shares Outstanding']

Define the data types dictionary for the columns
```
dt_setup = {
    'Ticker':'category',
    'Open':'float32',
    'High':'float32',
    'Low':'float32',
    'Close':'float32'
}
```

In [ ]:
%%time
new_shares = pd.read_csv('US_Share_Prices_2022.csv', delimiter=',', usecols=required_cols, dtype=dt_setup, parse_dates=[2])
len(shares)


In [ ]:
new_shares.info()

The new dataframe with the filtered data has been pulled in correctly.

In [ ]:
new_shares.head(10)

In [ ]:
new_shares.describe()

Check the dates:

In [ ]:
stock_data_2022['Date'].agg(['min', 'max'])

### Future Code for bringing in a updated CSV from SimFin
From the analysis of the existing data, I will use defined required columns and data types to create a new file to work from that will also pull only the filtered data for 2022.

In [ ]:
# Define the required columns for the data
required_cols = ['Ticker', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Shares Outstanding']

In [ ]:
# Define the data types for the columns
dt_setup = {
    'Ticker':'string',
    'Open':'float32',
    'High':'float32',
    'Low':'float32',
    'Close':'float32'
}

In [ ]:
%%time
# Use nrows option
shares = pd.read_csv('us-shareprices-daily.csv', delimiter=';', nrows=1000, usecols=required_cols, dtype=dt_setup, date_parse(2))
len(shares)


In [ ]:
# Fill na to zero the column, Shares Outstanding 
# shares = shares.fillna(0)

Code for converting dataframe to JSON and compressed JSON:

In [ ]:
# shares.to_json('US_Share_Prices_2022.json', orient='records', lines=True)

In [ ]:
# shares.to_json('US_Share_Prices_2022.json.gz', orient='records', lines=True, compression='gzip')

In [ ]:
# List out dates
shares['Date'].dt.date

In [ ]:
data_for_desired_year = shares[shares['Date'].dt.year == 2022]


In [ ]:
print(data_for_desired_year)